In [ ]:
import os
import re
import gc
import numpy as np
import pandas as pd
import random

from scipy.stats.mstats import winsorize
from sklearn.preprocessing import LabelEncoder
from scipy import stats
import tensorflow as tf

gc.enable()
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [ ]:
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed) 
random.seed(seed) 
tf.random.set_seed(seed) 

In [ ]:
data = pd.read_csv('/kaggle/input/al-moutmir/data.csv')

## Filter Rows & Drop Columns

In [ ]:
# Drop rows where 'variety' equals "Irrigated"
data = data[data['water_regime'] != "Irrigated"]

# Drop rows where 'crop' equals "Oats"
data = data[data['crop'] != 'Oats']
# data = data[data['crop'] != 'Baley']

# Drop specified columns
data.drop(['variety', 'water_regime', 'weather_date_range', 'growth_season_year'], axis=1, inplace=True)

In [ ]:
# Filter out future-collected columns
future_collected_columns = [
    'ID', 'harvesting_date', 'sowing_date', 'sowing_rate_kg_ha', 'height_at_anthesis_cm',
    'plant_density_tillering', 'tillers_at_booting', 'ears_per_plant', 'fresh_biomass_kg_ha',
    'harvest_index', 'thousand_seed_weight_g', 'topdress_calcium_cao', 'topdress_potassium_k2o',
    'topdress_phosphorus_p2o5', 'topdress_magnesium_mgo', 'topdress_sulfur_so3', 'topdress_nitrogen',
    'topdress_applications'
]

data.drop(future_collected_columns, axis=1, inplace=True, errors='ignore')

## Clean Numeric and Non–Numeric Columns

In [ ]:
region_mapping = {
    'Beni mellal - Khénifra ': 'Beni Mellal-Khénifra',
    'Oriental ': 'Oriental',
    'Fès - Meknès': 'Fès-Meknès',
    'Marrakach-safi': 'Marrakech-Safi',
    'Rabat Sale Kénita': 'Rabat-Salé-Kénitra',
    'Tanger-Tétouan-Al Hoceïma': 'Tanger-Tétouan-Al Hoceima',
    'Casablanca-Settat': 'Casablanca-Settat',
    'Draa-Tafilalet': 'Drâa-Tafilalet'
}

# Clean and map region names
data['region'] = data['region'].map(region_mapping)

In [ ]:
def clean_province_names(data, column_name):
    # Standardize names: remove extra spaces, handle casing
    data[column_name] = data[column_name].str.strip().str.title()

    # Define mappings for specific corrections
    province_corrections = {
        "El jadida": "El Jadida",
        "Fés": "Fès",
        "Khémisset": "Khemisset",
        "Khénifra": "Khenifra",
        "Séfrou": "Sefrou",
        "Sidi bennour": "Sidi Bennour",
        "Azilal ": "Azilal",
        "Fkih ben salah": "Fkih Ben Salah",
        "OUEZZANE": "Ouezzane",
        "Sidi slimane": "Sidi Slimane",
        "TANGER ASSILAH": "Tanger-Assilah",
        "ASSILAH": "Assilah",
        "El Youssoufia": "Youssoufia"
    }

    # Apply corrections
    data[column_name] = data[column_name].replace(province_corrections)

    return data

# Apply the function to the 'province' column
data = clean_province_names(data, 'province')

In [ ]:
# Define a function to clean the 'previous_crop' column
def clean_previous_crop(data, column_name):
    # Standardize the column to lowercase
    data[column_name] = data[column_name].str.lower().str.strip()

    # Define mappings for grouping similar categories
    crop_mapping = {
        "soft wheat": "Soft Wheat",
        "fallow": "Fallow",
        "chickpea": "Chickpea",
        "sugar beet": "Sugar Beet",
        "bean": "Bean",
        "beans": "Bean",
        "faba bean": "Faba Bean",
        "dry beans": "Dry Bean",
        "durum wheat": "Durum Wheat",
        "lens": "Lens",
        "barley": "Barley",
        "oats": "Oats",
        "onion": "Onion",
        "potato": "Potato",
        "tomato (open field)": "Tomato",
        "orobe": "Orobe",
        "melon": "Melon",
        "alfalfa": "Alfalfa",
        "fennel flower": "Fennel Flower",
        "triticale": "Triticale",
        "watermelon": "Watermelon",
        "rapeseed": "Rapeseed",
        "sesame": "Sesame",
        "other veg": "Other Vegetables",
        "pea": "Pea",
        "sugar cane": "Sugar Cane",
        "niora": "Niora",
        "parsley": "Parsley",
        "cress": "Cress",
        "fodder crop": "Fodder Crop",
        "cordiander": "Coriander",
        "carrot": "Carrot",
        "cabbage": "Cabbage"
    }

    # Map the cleaned values using the defined mapping
    data[column_name] = data[column_name].map(crop_mapping).fillna(data[column_name])

    # Standardize final output to title case for uniformity
    data[column_name] = data[column_name].str.title()

    return data

# Apply the cleaning function to the 'previous_crop' column
data = clean_previous_crop(data, 'previous_crop')
data['previous_crop'].fillna('Other Vegetables', inplace=True)

In [ ]:
# Clean the 'crop' column
data['crop'] = data['crop'].replace({
    'S. wheat': 'Soft Wheat',
    'D. Wheat': 'Durum Wheat',
    'Baley': 'Barley'
})

## Clean Numeric Columns

In [ ]:
# Define a function to clean numeric columns
def clean_numeric_columns(df, exclude_substrings):
    # Automatically detect numeric columns
    numeric_columns = [col for col in df.select_dtypes(include=['object', 'float', 'int']).columns]

    # Exclude columns containing any of the specified substrings
    columns_to_clean = [
        col for col in numeric_columns 
        if not any(substring in col for substring in exclude_substrings)
    ]

    # Replace specific unwanted strings and clean the data
    df[columns_to_clean] = df[columns_to_clean].replace([' -   ', ' -     ', ' _ '], '')

    # Remove commas, strip whitespace, and convert to numeric
    for col in columns_to_clean:
        df[col] = df[col].astype(str).str.replace(',', '', regex=False).str.strip()
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

# Exclude substrings
exclude_substrings = [
    "growth_season", "province", "region", "crop", "sub_program"
]

# Example usage with your DataFrame 'data'
data = clean_numeric_columns(data, exclude_substrings)

## Label Encoding

In [ ]:
# Columns to label encode
label_encode_columns = ['growth_season', 'region', 'province', 'sub_program',
                        'crop', 'previous_crop'] #location_id

# Initialize LabelEncoders for each column
label_encoders = {col: LabelEncoder() for col in label_encode_columns}

# Apply LabelEncoder to each column
for col in label_encode_columns:
    data[col] = label_encoders[col].fit_transform(data[col])

In [ ]:
# Display the label encoding mappings for each column
for col, encoder in label_encoders.items():
    print(f"Label encoding for '{col}':")
    mapping = {original: encoded for encoded, original in enumerate(encoder.classes_)}
    print(mapping)

## Drop Missing Values

In [ ]:
# List of substrings to exclude
exclude_substrings = None

# Filter columns to exclude those containing specified substrings
columns_to_exclude = [col for col in data.columns if any(substring in col for substring in exclude_substrings)]
filtered_df = data.drop(columns=columns_to_exclude)

# Calculate and sort the percentage of missing values for the remaining columns
missing_pct = filtered_df.isnull().mean() * 100
missing_pct_sorted = missing_pct.sort_values(ascending=False)

# Print the sorted percentages of missing values
print("Percentage of missing values per column (sorted, filtered):")
missing_pct_sorted

In [ ]:
# Dropping columns with more than 30% missing values
# Define a threshold for dropping columns (e.g., 30% missing values)
threshold = 0.3

# List columns to be dropped based on missing percentage threshold
dropped_columns = []

for col in data.columns:
    missing_percentage = data[col].isnull().sum() / len(data)
    
    # Check if column has more than the threshold of missing values
    if missing_percentage > threshold:
       dropped_columns.append(col)

# Drop columns that meet the threshold and are not deemed essential
data.drop(dropped_columns, axis=1, inplace=True)

## Winsorization

In [ ]:
# Columns to Winsorize
columns_to_winsorize = [
    'soil_ph', 'organic_matter_percent', 'phosphorus_ppm', 'potassium_ppm',
    'electrical_conductivity', 'npk_nitrogen', 'npk_phosphorus_p2o5',
    'npk_potassium_k2o', 'npk_magnesium_mgo', 'npk_calcium_cao',
    'grain_yield_kg'
]

# Dictionary to store Winsorization limits
limits_dict = {}

# Winsorize each specified column and record limits
for column in columns_to_winsorize:
    if column in data.columns:
        # Calculate the proportion of data trimmed on both ends
        q1 = data[column].quantile(0.15)
        q9 = data[column].quantile(0.85)
        lower_limit = (data[column] < q1).mean()
        upper_limit = (data[column] > q9).mean()
        limits_dict[column] = (lower_limit, upper_limit)
        
        # Apply Winsorization
        data[column] = winsorize(data[column], limits=(lower_limit, upper_limit))

limits_dict

## Impute Missing Values

In [ ]:
# List of low-missingness features
low_missing_features = [
    'npk_magnesium_mgo', 'npk_calcium_cao', 'soil_ph',
    'organic_matter_percent', 'phosphorus_ppm', 'potassium_ppm',
    'npk_potassium_k2o', 'npk_phosphorus_p2o5', 'npk_nitrogen',
    'electrical_conductivity'
]

# Define group levels for median imputation (from most granular to least granular)
group_levels = [
    ['longitude', 'latitude', 'crop', 'previous_crop'],
    ['longitude', 'latitude', 'crop'],
    ['longitude', 'latitude'],
    ['province', 'crop', 'previous_crop'],
    ['province', 'crop']
]

# Function to perform grouped median imputation
def grouped_median_imputation(df, columns, groupings):
    """
    Perform grouped median imputation on specified columns.

    Parameters:
        df (pd.DataFrame): The DataFrame to modify.
        columns (list): List of columns to impute.
        groupings (list): List of grouping combinations (list of lists).
    """
    for groups in groupings:
        for col in columns:
            df[col] = df[col].fillna(df.groupby(groups)[col].transform('median'))
    return df

# Perform grouped median imputation
data = grouped_median_imputation(data, low_missing_features, group_levels)

# Fallback: Global median imputation for any remaining missing values
for col in low_missing_features:
    data[col].fillna(data[col].median(), inplace=True)

In [ ]:
label_encoder = LabelEncoder()
data['electrical_conductivity_encoded'] = label_encoder.fit_transform(data['electrical_conductivity'])

## Remove Outliers

In [ ]:
# List of crop types
crop_types = data['crop'].unique()

# Create a copy of the original dataset to avoid overwriting it
data_cleaned = data.copy()

# Loop over each crop type and remove outliers based on Z-score for each crop type individually
for crop in crop_types:
    # Filter the data for the current crop
    crop_data = data_cleaned[data_cleaned['crop'] == crop]
    
    # Calculate Z-scores only for the target column (grain_yield_kg) for this crop
    if len(crop_data) > 0:  # Ensure there are rows to avoid errors
        z_scores = np.abs(stats.zscore(crop_data['grain_yield_kg']))
        
        # Define outliers based on Z-score threshold
        # Z > 2.0 (95.4% of data falls within this range)
        outliers = z_scores > 2.0
        
        # Remove outliers for this crop type
        data = data_cleaned[~((data_cleaned['crop'] == crop) & outliers)]